In [1]:
import cv2
import numpy as np

# Define the colors for the chips
colors = {
    "robot": (255, 0, 0),  # Blue for robot's chip
    "opponent": (0, 255, 0)  # Green for opponent's chip
}

# Replace with your robot's camera URL
camera_url = 'http://<robot_ip>:<port>/video'  # Update with the correct URL

# Capture video from the robot's camera
cap = cv2.VideoCapture(camera_url)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to HSV color space for better color detection
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define color ranges for the chips
    lower_blue = np.array([100, 150, 0])  # Adjust for robot's chip
    upper_blue = np.array([140, 255, 255])
    
    lower_green = np.array([40, 150, 0])  # Adjust for opponent's chip
    upper_green = np.array([80, 255, 255])

    # Create masks for each color
    mask_robot = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_opponent = cv2.inRange(hsv, lower_green, upper_green)

    # Find contours for robot's chip
    contours_robot, _ = cv2.findContours(mask_robot, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_robot:
        area = cv2.contourArea(contour)
        if area > 100:  # Minimum area to filter out noise
            ((x, y), radius) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x), int(y)), int(radius), colors["robot"], 2)

    # Find contours for opponent's chip
    contours_opponent, _ = cv2.findContours(mask_opponent, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_opponent:
        area = cv2.contourArea(contour)
        if area > 100:  # Minimum area to filter out noise
            ((x, y), radius) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x), int(y)), int(radius), colors["opponent"], 2)

    # Display the results
    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms

# Load the pretrained DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

# Define transformations for the input
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((480, 640)),  # Adjust size as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define a mapping for colors (you can modify as needed)
colors = {
    21: (255, 0, 0),  # Example for "robot" (blue)
    22: (0, 255, 0),  # Example for "opponent" (green)
}

# Capture video from the robot's camera
camera_url = 'http://<robot_ip>:<port>/video'  # Update with your robot's camera URL
cap = cv2.VideoCapture(camera_url)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for the model
    input_tensor = preprocess(frame)
    input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

    # Run the model and get the output
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0).byte().cpu().numpy()

    # Create a color mask for the segmented output
    segmented_frame = np.zeros_like(frame)
    for label, color in colors.items():
        segmented_frame[output_predictions == label] = color

    # Combine the original frame with the segmented output
    combined_frame = cv2.addWeighted(frame, 0.5, segmented_frame, 0.5, 0)

    # Display the results
    cv2.imshow('Segmented Frame', combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
